In [20]:
import asyncio
import time
import os
from dataclasses import dataclass
from typing import Literal, Union
from pydantic import BaseModel, Field
from google import genai
from google.genai import types

from dotenv import load_dotenv
load_dotenv('../envs/.env', override=True)

client = genai.Client()


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


### Streaming + structured output - DOESNT WORK!



In [21]:
class QuizItemAnswer(BaseModel):
    answer: str = Field(description="The answer text")
    explanation: str = Field(description="The explanation text")
    correct: bool = Field(description="Whether the answer is correct")


class QuizItemTest(BaseModel):
    question: str = Field(description="The quiz question text")
    answers: list[QuizItemAnswer] = Field(
        description="4 answers with only one correct",
        min_length=4,
        max_length=4
    )


class TestOutput(BaseModel):
    mode: Literal["quiz"] = "quiz"
    quiz_items: list[QuizItemTest] = Field(
        description="Array of quiz items",
        min_length=5,
        max_length=5
    )

    
@dataclass
class TestDeps:
    """Dependencies for the test agent"""
    query: str


In [ ]:

from dotenv import load_dotenv
load_dotenv('../envs/.env', override=True)

client = genai.Client()

with open('image.png', 'rb') as f:
    image_bytes = f.read()


response = client.models.generate_content(
        model='gemini-2.5-flash-lite',
        contents=[
            types.Part.from_bytes(
                data=image_bytes,
                mime_type='image/png',
            ),
            'Transform the information (visual and textual) from the image into detailed text. Make it sound just like a factoid from the presentation. Dont notice any pictures in the output, just the facts! '
        ]
    )

print(response.text)

The image depicts a finite state machine for RDT 2.1 receiver, which handles garbled ACK/NAKs. It has two states: "Wait for 0 from below" and "Wait for 1 from below".

**From "Wait for 0 from below" state:**
*   **On receiving a packet (rcvpkt) that is not corrupt and has sequence number 0:**
    *   Extract data from the packet.
    *   Deliver the data.
    *   Create an ACK packet with the checksum.
    *   Send the ACK packet.
    *   Transition to "Wait for 1 from below".
*   **On receiving a packet (rcvpkt) that is corrupt:**
    *   Create a NAK packet with the checksum.
    *   Send the NAK packet.
    *   Stay in "Wait for 0 from below".
*   **On receiving a packet (rcvpkt) that is not corrupt but has sequence number 1:**
    *   Create an ACK packet with the checksum.
    *   Send the ACK packet.
    *   Stay in "Wait for 0 from below".

**From "Wait for 1 from below" state:**
*   **On receiving a packet (rcvpkt) that is not corrupt and has sequence number 1:**
    *   Extrac

In [24]:
print("🚀 Starting Gemini models comparison...\n")
print("="*60)

with open('image.png', 'rb') as f:
    image_bytes = f.read()

models = [
    'gemini-2.0-flash-lite',
    'gemini-2.5-flash-lite',
    'gemini-2.0-flash',
]

results = {}

for model_name in models:
    print(f"\n🔬 Testing {model_name}...")
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model_name,
        contents=[
            types.Part.from_bytes(
                data=image_bytes,
                mime_type='image/png',
            ),
            'Tell me please what is depicted in the image'
        ]
    )
    
    latency = time.time() - start_time
    
    print(f"  ⏱️  Latency: {latency:.2f}s")
    print(f"\n  📝 Response:")
    print(f"  {response.text}\n")
    print("-" * 60)


🚀 Starting Gemini models comparison...


🔬 Testing gemini-2.0-flash-lite...
  ⏱️  Latency: 1.48s

  📝 Response:
  The image illustrates the weakness of the Internet checksum in protecting data integrity. It shows an example where two 16-bit integers are added, and the result is used to calculate the checksum.  The diagram highlights that even after some bits in the original numbers are flipped (changed from 0 to 1 or vice versa), the checksum remains unchanged. This demonstrates that the checksum is not robust enough to detect certain types of errors or modifications in the data, thus highlighting its "weak protection".


------------------------------------------------------------

🔬 Testing gemini-2.5-flash-lite...
  ⏱️  Latency: 1.48s

  📝 Response:
  The image illustrates the weakness of the Internet checksum in protecting data integrity. It shows an example where two 16-bit integers are added, and the result is used to calculate the checksum.  The diagram highlights that even afte